In [10]:
import os
import shutil
import numpy as np
import cv2
from ultralytics import YOLO
from yolov8.dataset import Dataset
from yolov8.utils import load_yolo_weights
from yolov8.configs import *
from evaluate_mAP import get_mAP
import torch.onnx
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

YOLO_TYPE = "yolov8"
TRAIN_YOLO_TINY = True

if YOLO_TYPE == "yolov8":
    Darknet_weights = YOLO_V8_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V8_WEIGHTS
elif YOLO_TYPE == "yolov8":
    Darknet_weights = YOLO_V8_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V8_WEIGHTS
if TRAIN_YOLO_TINY:
    TRAIN_MODEL_NAME += "_Tiny"

    def main():
        global TRAIN_FROM_CHECKPOINT

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if torch.cuda.is_available():
        try:
            torch.cuda.set_memory_allocated(device, True)
        except RuntimeError:
            pass

    if os.path.exists(TRAIN_LOGDIR):
        shutil.rmtree(TRAIN_LOGDIR)
    writer = SummaryWriter(TRAIN_LOGDIR)
train_dataset = YourDataset('C:/Users/Haroon/Desktop/Fortnite_clean_dataset/train', transform=transforms.Compose([...]))  
test_dataset = YourDataset('C:/Users/Haroon/Desktop/Fortnite_clean_dataset/test', transform=transforms.Compose([...])) 

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

steps_per_epoch = len(train_loader)
global_steps = 1
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

if TRAIN_TRANSFER:
    Darknet = YourYoloModel(input_size=YOLO_INPUT_SIZE, num_classes=NUM_CLASSES)
    Darknet.load_darknet_weights(Darknet_weights)

yolo = YourYoloModel(input_size=YOLO_INPUT_SIZE, num_classes=NUM_CLASSES)
if TRAIN_FROM_CHECKPOINT:
    try:
        yolo.load_state_dict(torch.load(TRAIN_FROM_CHECKPOINT))
    except ValueError:
        print("Shapes are incompatible, transferring Darknet weights")
        TRAIN_FROM_CHECKPOINT = False

if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, (name, layer) in enumerate(Darknet.named_children()):
        if hasattr(layer, 'weight'):
            layer_weights = layer.weight
            if layer_weights.numel() > 0:
                try:
                    yolo_state_dict = yolo.state_dict()
                    yolo_state_dict[name + '.weight'] = layer_weights
                    yolo.load_state_dict(yolo_state_dict)
                except Exception as e:
                    print("Skipping layer {}: {}".format(name, str(e)))

optimizer = optim.Adam(yolo.parameters())

criterion_giou = nn.MSELoss()  
criterion_conf = nn.BCELoss()  
criterion_prob = nn.BCELoss()  

def train_step(image_data, target):
    optimizer.zero_grad()  

    pred_result = yolo(image_data)  

    giou_loss = conf_loss = prob_loss = 0

    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):
        conv, pred = pred_result[i * 2], pred_result[i * 2 + 1]
        loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)

        giou_loss += criterion_giou(loss_items[0], torch.zeros_like(loss_items[0]))
        conf_loss += criterion_conf(loss_items[1], torch.zeros_like(loss_items[1]))
        prob_loss += criterion_prob(loss_items[2], torch.zeros_like(loss_items[2]))

    total_loss = giou_loss + conf_loss + prob_loss

    total_loss.backward()  
    optimizer.step()
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=total_steps - warmup_steps, eta_min=TRAIN_LR_END)

global_steps += 1
if global_steps < warmup_steps:
    lr = global_steps / warmup_steps * TRAIN_LR_INIT
else:
    lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END) * (1 + torch.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * 3.14159265359))
lr_scheduler.step(global_steps)
writer = SummaryWriter(TRAIN_LOGDIR)

with writer.as_default():
    writer.add_scalar("lr", optimizer.param_groups[0]['lr'], global_steps)
    writer.add_scalar("loss/total_loss", total_loss, global_steps)
    writer.add_scalar("loss/giou_loss", giou_loss, global_steps)
    writer.add_scalar("loss/conf_loss", conf_loss, global_steps)
    writer.add_scalar("loss/prob_loss", prob_loss, global_steps)
writer.close()
validate_writer = SummaryWriter(TRAIN_LOGDIR + "_validate")

with validate_writer.as_default():
    writer.add_scalar("validate_loss/giou_val", giou_loss/count, epoch)
    writer.add_scalar("validate_loss/conf_val", conf_loss/count, epoch)
    writer.add_scalar("validate_loss/prob_val", prob_loss/count, epoch)
    writer.add_scalar("validate_loss/total_val", total_loss/count, epoch)
validate_writer.close()

mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
best_val_loss = 1000

writer = SummaryWriter(TRAIN_LOGDIR)

for epoch in range(TRAIN_EPOCHS):
    for image_data, target in trainset:
        results = train_step(image_data, target)
        cur_step = results[0] % steps_per_epoch
        print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
              .format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))

    if len(testset) == 0:
        print("configure TEST options to validate model")
        torch.save(mAP_model.state_dict(), os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME))
        continue

    count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
    for image_data, target in testset:
        results = validate_step(image_data, target)
        count += 1
        giou_val += results[0]
        conf_val += results[1]
        prob_val += results[2]
        total_val += results[3]

    with writer.as_default():
        writer.add_scalar("validate_loss/total_val", total_val / count, epoch)
        writer.add_scalar("validate_loss/giou_val", giou_val / count, epoch)
        writer.add_scalar("validate_loss/conf_val", conf_val / count, epoch)
        writer.add_scalar("validate_loss/prob_val", prob_val / count, epoch)

    writer.flush()

    print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
          format(giou_val / count, conf_val / count, prob_val / count, total_val / count))
    
    if TRAIN_SAVE_CHECKPOINT and not TRAIN_SAVE_BEST_ONLY:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME + "_val_loss_{:7.2f}".format(total_val / count))
    torch.save(yolo.state_dict(), save_directory)
if TRAIN_SAVE_BEST_ONLY and best_val_loss > total_val / count:
    save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
    torch.save(yolo.state_dict(), save_directory)
    best_val_loss = total_val / count
if not TRAIN_SAVE_BEST_ONLY and not TRAIN_SAVE_CHECKPOINT:
    save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
    torch.save(yolo.state_dict(), save_directory)

try:
    mAP_model.load_state_dict(torch.load(save_directory))
    mAP = get_mAP(mAP_model, testset, score_threshold=TEST_SCORE_THRESHOLD, iou_threshold=TEST_IOU_THRESHOLD)
    with writer.as_default():
        writer.add_scalar("validate_mAP", mAP, epoch)
    writer.flush()
except FileNotFoundError:
    pass

if __name__ == '__main__':
    main()
    
onnx_filename = "yolov8.onnx" 
dummy_input = torch.randn(1, 3, YOLO_INPUT_SIZE, YOLO_INPUT_SIZE).to(device)
torch.onnx.export(yolo, dummy_input, onnx_filename, verbose=True, input_names=['input'], output_names=['output'])

NameError: name 'YOLO_V8_TINY_WEIGHTS' is not defined